In [1]:
import numpy as np  # NumPy: For numerical and array operations.
import pandas as pd  # Pandas: For data manipulation and analysis.
import matplotlib.pyplot as plt  # Matplotlib: For creating various types of plots and charts.
import seaborn as sns  # Seaborn: For making data visualizations more attractive and informative.
import re 
import glob

In [2]:
site_visit = pd.read_csv("../data/RAW/google_analytic_qr_scan.csv")

site_visit['Date'] = pd.to_datetime(site_visit['Date'], format='%Y%m%d')
site_visit['Date'] = site_visit['Date'].dt.strftime('%Y/%m/%d')


site_visit['Page location'] = site_visit['Page location'].astype(str)

Identify which group they are from

- DM: 1
- List A + List C = Trifold	= airfryer/ airfryer3/ airfryer5"
- List B + List D =	Envelope = airfryer2/ airfryer4

- DM2 : List A = Postcard = airfryer6

In [3]:
def categorize_drop(row):
    lowered_row = row.lower()
    if "airfryer2" in lowered_row or "airfryer4" in lowered_row:
        return "DM1_envelope"
    elif "airfryer3" in lowered_row or "airfryer5" in lowered_row:
        return "DM1_trifold"
    elif "airfryer6" in lowered_row:
        return "DM2_postcard"
    elif "airfryer" in lowered_row:
        return "DM1_trifold"  # Update with the appropriate category for general "airfryer"
    else:
        return None


# Apply the function to create the 'Category' column
site_visit['Category'] = site_visit['Page location'].apply(categorize_drop)

site_visit['Airfryer_Type'] = site_visit['Page location'].str.extract(r'(airfryer\d?)', flags=re.IGNORECASE)

In [4]:
print(site_visit['Page location'].nunique())

119


In [5]:
## getting the id for site visit (scanned qr code, submitted the forms)

site_visit['ID'] = site_visit['Page location'].str.extract(r'utm_id=(\d+)')
site_visit.dropna(subset=['ID'], inplace=True)

site_visit = site_visit[site_visit['ID']!='00000']

id_counts = site_visit['ID'].value_counts()
site_visit['visit_again'] = site_visit['ID'].map(id_counts) > 1
site_visit['visit_again'] = site_visit['visit_again'].fillna(False)

print(site_visit['ID'].nunique())

site_visit.to_csv("../data/1site_visit_draft.csv")

110


In [6]:
site_visit_unique_ids = site_visit.drop_duplicates(subset=['ID'], keep='last')

In [7]:

form_submit = pd.read_csv("../data/RAW/submissions.csv")
form_submit['ID'] = form_submit['Referrer'].str.extract(r'utm_id=(\d+)')
form_submit['utm_id'].fillna(form_submit['Referrer'].str.extract(r'utm_id=(\d+)').iloc[:, 0].astype(float), inplace=True)

form_submit.rename(columns={'姓氏': 'Name',  '手机号码': 'Phone'}, inplace=True)


form_submit['Category'] = form_submit['Referrer'].apply(categorize_drop)

form_submit['Airfryer_Type'] = form_submit['Referrer'].str.extract(r'(airfryer\d?)', flags=re.IGNORECASE)
form_submit['Phone'] = form_submit['Phone'].str.replace('-', '', regex=True)

In [8]:
# Extract IDs from form_submit as a list
form_submit2 = form_submit.dropna(subset=['utm_id'])

extracted_ids = form_submit2['utm_id'].tolist()

formatted_ids = [str(int(id)).zfill(5) if isinstance(id, (int, float)) else id for id in extracted_ids]

# Use .loc to avoid the SettingWithCopyWarning
site_visit_unique_ids.loc[:, 'form_submission'] = site_visit_unique_ids.loc[:, 'ID'].isin(formatted_ids).astype(str)


C:\Users\skt\AppData\Local\Temp\ipykernel_24972\2116837197.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_visit_unique_ids.loc[:, 'form_submission'] = site_visit_unique_ids.loc[:, 'ID'].isin(formatted_ids).astype(str)


In [9]:
site_visit_unique_ids=site_visit_unique_ids.drop(columns = ['Sessions'])

In [10]:
site_visit_unique_ids.to_csv("../data/Clean/unique_qr_scan_nov.csv")